# Collision checking tutorial 💥

This notebook shows how to:
* Set up collision checking for joint configurations
* Apply collision checking to joint and TCP paths

## 1. Creating a `SceneGraphCollisionChecker` 🛡️
Drake provides a very convenient class called [`SceneGraphCollisionChecker`](https://drake.mit.edu/doxygen_cxx/group__planning__collision__checker.html) that checks for collisions between the robot's links itself and its environment.
After you set it up with a `RobotDiagram`, you can check for collisions with:
```python
collision_checker.CheckConfigCollisionFree(q)
```
Where `q` are all the joint positions of the `MultibodyPlant`.

In the previous notebooks we showed how to quickly create a `RobotDiagram` with a UR5e and Robotiq 2F-85 gripper.
You might have noticed that gripper introduces 6 degrees of freedom.
However, in this notebook we will, for simplicity, make the gripper static and do collision checks with the gripper in a fixed open position.

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import add_floor, add_meshcat, finish_build, X_URTOOL0_ROBOTIQ, X_URBASE_ROSBASE
from airo_drake import SingleArmScene
from pydrake.math import RigidTransform
import airo_models

robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
add_floor(robot_diagram_builder)

plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
parser.SetAutoRenaming(True)

# Load URDF files
arm_name = "ur5e"
gripper_name = "robotiq_2f_85"
arm_urdf_path = airo_models.get_urdf_path(arm_name)
gripper_urdf_path = airo_models.get_urdf_path(gripper_name)

# Make the gripper static
gripper_urdf = airo_models.urdf.read_urdf(gripper_urdf_path)
airo_models.urdf.replace_value(gripper_urdf, "@type", "revolute", "fixed")
airo_models.urdf.delete_key(gripper_urdf, "mimic")
airo_models.urdf.delete_key(gripper_urdf, "transmission")
gripper_static_urdf_path = airo_models.urdf.write_urdf_to_tempfile(
    gripper_urdf, gripper_urdf_path, prefix=f"{gripper_name}_static_"
)

# Use static gripper
arm_index = parser.AddModels(arm_urdf_path)[0]
gripper_index = parser.AddModels(gripper_static_urdf_path)[0]

# Weld some frames together
world_frame = plant.world_frame()
arm_frame = plant.GetFrameByName("base_link", arm_index)
arm_tool_frame = plant.GetFrameByName("tool0", arm_index)
gripper_frame = plant.GetFrameByName("base_link", gripper_index)

arm_rigid_transform = X_URBASE_ROSBASE
gripper_rigid_transform = X_URTOOL0_ROBOTIQ

plant.WeldFrames(world_frame, arm_frame, arm_rigid_transform)
plant.WeldFrames(arm_tool_frame, gripper_frame, gripper_rigid_transform)

robot_diagram, context = finish_build(robot_diagram_builder, meshcat)
del robot_diagram_builder  # no longer needed

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)
scene

In [ ]:
plant.num_positions(), plant.num_positions(arm_index), plant.num_positions(gripper_index)

In [ ]:
from pydrake.planning import SceneGraphCollisionChecker

collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[scene.arm_index, scene.gripper_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

## 2. Checking joint configurations 🤖

Lets test some joint configurations for collisions!

First, the default configuration. You can see in Meshcat that the robot is just barely not colliding with the table.

In [ ]:
collision_checker.CheckConfigCollisionFree(np.zeros(6))

UR robots are always in self-collsion when elbow joint is at 180 degrees:

In [ ]:
joints_self_collision = np.deg2rad([0, 0, 180, 0, 0, 0])
collision_checker.CheckConfigCollisionFree(joints_self_collision)

Lets double-check that visually:

In [ ]:
plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, scene.arm_index, joints_self_collision)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

We can make the robot collide with the table by make the shoulder angle slightly larger

In [ ]:
joints_table_collision = np.deg2rad([0, 5, 0, 0, 0, 0])
collision_checker.CheckConfigCollisionFree(joints_table_collision)

In [ ]:
plant.SetPositions(plant_context, scene.arm_index, joints_table_collision)
scene.robot_diagram.ForcedPublish(context)

## Checking joint paths 🛤️